In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
directory = "./Logs/"
save_path = "./Saved/"

from load_logs import *
from evaluation import *
from features import *
from visualize import *

logs, log_labels = read_logs_and_labels("./Saved/logs.txt", "./Saved/labels.txt")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import os
import pickle
import sys
import random
import hdbscan

from scipy import sparse
from math import sqrt
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [7]:
#Time to define necessary functions
#Since this is based on a knn idea we will need to calculate euclidean distance

In [8]:
#Euclidan distance function
def eucDist(x,y):
    dist = 0.0
    for i in range (len(y)-1):
        dist += (x[i] - y[i])**2
    return sqrt(dist)

In [10]:
#Nearest neighbour 
def nearestNeigh(x, test_row, neighbors): #specify number of neighbours to return
    dist = list()
    for x_row in x:
        dist = euclidean_distance(y_row, x_row)
        distances.append((x_row, dist))
        distances.sort(key=lambda tup: tup[1])
    neighborsList = list()
    for i in range(neighbors):
        neighborsList.append(dist[i][0])
    return neighborsList

In [5]:
labels_ = log_labels
            
X = get_features(logs, 2, 70)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)
## K Means Labels
kmeans = KMeans(init="k-means++", n_clusters=10,random_state=0).fit(X)
kLabels = kmeans.labels_

## DBSCAN Labels
DBmodel = DBSCAN(eps=0.4,min_samples=200,algorithm="auto",metric="cosine")
DBmodel.fit(X)
dbLabels = DBmodel.labels_

## HDBScan Label
Hmodel = hdbscan.HDBSCAN(min_cluster_size=500,min_samples=100,cluster_selection_epsilon=0.5,
                                        metric="euclidean",algorithm='best')
Hmodel.fit(X)
hdcLabels = Hmodel.labels_